Final Project

Paul Astorga & Kayden Finlayson

We are going to build a few models which attempt to predict whether or not an employee is granted access to a resource given data on each employee. Our data is Amazon employee access data which we retrieved from Kaggle. Let's start by importing some modules to help us build out our models, then we will load in our data and see what we can learn.

In [1]:
import pandas as pd
import seaborn as sns
import csv
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import seaborn as sns
import sqlite3
%matplotlib inline
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

## Load in Data

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

## Column Name	Description

### Identification

- RESOURCE:	An ID for each resource
- MGR_ID: The EMPLOYEE ID of the manager of the current EMPLOYEE ID record; an employee may have only one manager at a time

### Dependent Variable

- ACTION: is 1 if the resource was approved, 0 if the resource was not


### Independent Variables

- ROLE_ROLLUP_1:	Company role grouping category id 1 (e.g. US Engineering)
- ROLE_ROLLUP_2:	Company role grouping category id 2 (e.g. US Retail)
- ROLE_DEPTNAME:	Company role department description (e.g. Retail)
- ROLE_TITLE:	Company role business title description (e.g. Senior Engineering Retail Manager)
- ROLE_FAMILY_DESC: Company role family extended description (e.g. Retail Manager, Software Engineering)
- ROLE_FAMILY:	Company role family description (e.g. Retail Manager)
- ROLE_CODE:	Company role code; this code is unique to each role (e.g. Manager)

## Understanding the Data

In [3]:
# Get the shape of the training and test sets

train_df_shape = list(df_train.shape)
train_df_shape[0] = str(train_df_shape[0]) + " Rows"
train_df_shape[1] = str(train_df_shape[1]) + " Columns"
print("Train", train_df_shape)

test_df_shape = list(df_test.shape)
test_df_shape[0] = str(test_df_shape[0]) + " Rows"
test_df_shape[1] = str(test_df_shape[1]) + " Columns"
print("Test", test_df_shape)

Train ['32769 Rows', '10 Columns']
Test ['912363 Rows', '10 Columns']


Good, we've got a lot of data to work with in our train and almost 1 million in our test set.  That means we can be confident that at least our modeling efforts won't be underpowered. 

In [4]:
df_train.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [5]:
df_test.head()

,id,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,32642,7792,118573,118574,117945,136261,128463,292795,119082
1,2,4696,14638,117961,118343,118514,118321,289122,255851,118322
2,3,22662,1760,118887,118888,120171,118396,255118,118398,118399
3,4,75078,7576,117961,118052,120671,118321,117906,257051,118322
4,5,39879,55668,117902,118041,117945,135951,134458,19776,119082


Taking a quick look at our training and test sets tells a few things. First the good news: every element is a number which makes this dataset literally plug and play from a modeling perspective. The bad news is that this makes it a little harder to conceptualize and understand what each element means. Some are easier to understand than others, for example there is a unique MGR_ID for each manager and unique ID for each RESOURCE.

Now we should check to see if we are missing any data in both our train and test sets.

In [6]:
df_train.isnull().sum().sort_values(ascending=False).head(5)

ROLE_CODE           0
ROLE_FAMILY         0
ROLE_FAMILY_DESC    0
ROLE_TITLE          0
ROLE_DEPTNAME       0
dtype: int64

In [7]:
df_test.isnull().sum().sort_values(ascending=False).head(5)

ROLE_CODE           0
ROLE_FAMILY         0
ROLE_FAMILY_DESC    0
ROLE_TITLE          0
ROLE_DEPTNAME       0
dtype: int64

Nice, we aren't missing any data!

In [8]:
cutoff = int(df_train.shape[0]/2)
x_train = df_train.loc[:(cutoff - 1), ~df_train.columns.isin(["ACTION"])]
y_train = df_train["ACTION"][:cutoff]
x_test = df_train.loc[cutoff:, ~df_train.columns.isin(["ACTION"])]
y_test = df_train["ACTION"][cutoff:]

In [9]:
x_train.shape

(16384, 9)

In [10]:
y_train.shape

(16384,)

In [11]:
x_test.shape

(16385, 9)

In [12]:
y_test.shape

(16385,)

## Models

In [13]:
from sklearn.linear_model import LogisticRegression 
from sklearn import metrics

Logistic = LogisticRegression()
Logistic.fit(x_train, y_train)
y_pred_train = Logistic.predict(x_train)

print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

Accuracy: 0.941650390625


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [14]:
Logistic = LogisticRegression()
Logistic.fit(x_train, y_train)
y_pred_test = Logistic.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))

Accuracy: 0.9425694232529753


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [15]:
x_train.head ()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [16]:
cutoff = int(df_train.shape[0]/2)
x_train = df_train.loc[:(cutoff - 1), ~df_train.columns.isin(["ACTION"])]
y_train = df_train["ACTION"][:cutoff]
x_test = df_train.loc[cutoff:, ~df_train.columns.isin(["ACTION"])]
y_test = df_train["ACTION"][cutoff:]

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(learning_rate=0.01, max_depth=1)
clf.fit(x_train,y_train)
y_pred_train = clf.predict(x_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

Accuracy: 0.941650390625


In [18]:
clf = GradientBoostingClassifier(learning_rate=0.01, max_depth=1)
clf.fit(x_train,y_train)
y_pred_test = clf.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))

Accuracy: 0.9425694232529753


In [19]:
cutoff = int(df_train.shape[0]/2)
x_train = df_train.loc[:(cutoff - 1), ~df_train.columns.isin(["ACTION"])]
y_train = df_train["ACTION"][:cutoff]
x_test = df_train.loc[cutoff:, ~df_train.columns.isin(["ACTION"])]
y_test = df_train["ACTION"][cutoff:]

In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, max_depth=5)
clf.fit(x_train,y_train)
y_pred_train = clf.predict(x_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

Accuracy: 0.9422607421875


In [21]:
clf = RandomForestClassifier(n_estimators=10, max_depth=5)
clf.fit(x_train,y_train)
y_pred_test = clf.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))

Accuracy: 0.9425083918217882


In [22]:
x_train = df_train.loc[:, ~df_train.columns.isin(["ACTION"])]
y_train = df_train["ACTION"]
x_test = df_test.loc[:, ~df_test.columns.isin(["id"])]
y_test = df_train["ACTION"]

In [23]:
clf = RandomForestClassifier(n_estimators=10, max_depth=5)
clf.fit(x_train,y_train)
y_pred_train = clf.predict(x_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

Accuracy: 0.9422319875492081


In [24]:
x_train["ACT"] = y_train
x_train["PRED"] = y_pred_train

In [25]:
x_train.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,ACT,PRED
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1,1


In [26]:
x_train[x_train["ACT"] != x_train["PRED"]]

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,ACT,PRED
5,45333,14561,117951,117952,118008,118568,118568,19721,118570,0,1
41,74310,49521,117961,118300,118301,119849,235245,118638,119851,0,1
55,78106,50613,117916,118150,118810,118568,159905,19721,118570,0,1
66,35043,14800,117961,117962,118352,118784,117906,290919,118786,0,1
68,15716,18073,118256,118257,118623,118995,286106,292795,118997,0,1
91,22956,3967,117961,118052,118706,118321,117906,290919,118322,0,1
102,36480,15510,118169,118170,118171,130479,185784,119784,130481,0,1
112,80501,19717,117961,117962,118352,118321,117906,290919,118322,0,1
122,23096,29762,117961,118052,120398,122067,130238,118424,122069,0,1
156,77977,3811,120864,121013,118395,118396,269406,118398,118399,0,1


In [27]:
x_train[x_train["ACT"] != x_train["PRED"]]["PRED"].value_counts()

1    1893
Name: PRED, dtype: int64

There are 1888 incorrect assignments

In [28]:
# Nelson's magic trick:

value_counts = x_train[x_train["ACT"] != x_train["PRED"]]["RESOURCE"].value_counts()
value_counts = pd.DataFrame(value_counts).reset_index()

totals = x_train["RESOURCE"].value_counts()
totals = pd.DataFrame(totals).reset_index()
totals = pd.merge(totals, value_counts, "left", left_on = "index", right_on = "index")
totals["Proportion"] = totals["RESOURCE_y"]/totals["RESOURCE_x"]
totals.sort_values("Proportion", ascending=False)


,index,RESOURCE_x,RESOURCE_y,Proportion
7475,40472,1,1.0,1.0
4856,106868,1,1.0,1.0
4669,114822,1,1.0,1.0
4686,44884,1,1.0,1.0
4694,26381,1,1.0,1.0
4703,16038,1,1.0,1.0
4704,80729,1,1.0,1.0
4710,82090,1,1.0,1.0
4711,80569,1,1.0,1.0
4737,16282,1,1.0,1.0


In [29]:
totals.sort_values(by=['Proportion'])

,index,RESOURCE_x,RESOURCE_y,Proportion
0,4675,839,3.0,0.003576
3,75078,409,4.0,0.009780
32,34950,91,1.0,0.010989
34,73756,87,1.0,0.011494
9,17308,239,3.0,0.012552
18,79121,157,2.0,0.012739
44,73815,73,1.0,0.013699
4,3853,404,6.0,0.014851
56,40867,62,1.0,0.016129
6,75834,299,5.0,0.016722


In [30]:
totals.to_sql('totals_database', con=engine)

In [85]:
engine.execute("SELECT * FROM totals_database WHERE RESOURCE_x >= 10 ORDER BY Proportion DESC").fetchall()

[(98, 27416, 37, 18.0, 0.4864864864864865),
 (439, 20287, 11, 5.0, 0.45454545454545453),
 (28, 20897, 104, 42.0, 0.40384615384615385),
 (228, 75216, 19, 6.0, 0.3157894736842105),
 (359, 20231, 13, 4.0, 0.3076923076923077),
 (455, 33233, 10, 3.0, 0.3),
 (472, 39631, 10, 3.0, 0.3),
 (112, 20222, 33, 9.0, 0.2727272727272727),
 (418, 33229, 11, 3.0, 0.2727272727272727),
 (421, 38205, 11, 3.0, 0.2727272727272727),
 (425, 44655, 11, 3.0, 0.2727272727272727),
 (448, 80501, 11, 3.0, 0.2727272727272727),
 (226, 20284, 19, 5.0, 0.2631578947368421),
 (139, 43877, 28, 7.0, 0.25),
 (378, 33239, 12, 3.0, 0.25),
 (381, 19751, 12, 3.0, 0.25),
 (390, 31183, 12, 3.0, 0.25),
 (154, 77300, 25, 6.0, 0.24),
 (206, 38124, 21, 5.0, 0.23809523809523808),
 (275, 79168, 17, 4.0, 0.23529411764705882),
 (178, 302049, 23, 5.0, 0.21739130434782608),
 (20, 18072, 136, 29.0, 0.21323529411764705),
 (172, 25749, 24, 5.0, 0.20833333333333334),
 (135, 24016, 29, 6.0, 0.20689655172413793),
 (53, 18913, 65, 13.0, 0.2),
 (48

This shows the resources that performed the worst with more than 10 observations

In [31]:
engine.execute("SELECT COUNT(RESOURCE_X) FROM totals_database WHERE RESOURCE_x = RESOURCE_y").fetchall()

[(292,)]

This is the number of times that the model was 100% incorrect in assignments 

In [32]:
engine.execute("SELECT COUNT(RESOURCE_x) FROM totals_database WHERE RESOURCE_x = 1").fetchall()

[(3766,)]

This is the total number of distinct resources with only one observation

In [33]:
engine.execute("SELECT COUNT(RESOURCE_x) FROM totals_database WHERE RESOURCE_x = 1 AND RESOURCE_y = 1").fetchall()

[(268,)]

This is the number of times that the distinct resources with only one observation was wrong

In [34]:
engine.execute("SELECT * FROM totals_database WHERE Proportion > 0.45 AND RESOURCE_x < 30 AND RESOURCE_x > 5").fetchall()

[(439, 20287, 11, 5.0, 0.45454545454545453),
 (614, 44976, 8, 7.0, 0.875),
 (684, 21751, 7, 7.0, 1.0),
 (890, 22956, 6, 5.0, 0.8333333333333334),
 (898, 20218, 6, 3.0, 0.5)]

In [35]:
engine.execute("SELECT COUNT(RESOURCE_x) FROM totals_database WHERE RESOURCE_x = 1").fetchall()

[(3766,)]

prop_totalwrong = 267/1888
print(prop_totalwrong)

In [36]:
unique_x_train = x_train["RESOURCE"].unique()
print(unique_x_train)

[39353 17183 36724 ... 32529 16354 32147]


In [38]:
x_train.head ()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,ACT,PRED
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1,1


In [39]:
x_train[x_train["ACT"] != x_train["PRED"]]["ACT"].value_counts()


0    1893
Name: ACT, dtype: int64

In [40]:
x_train[x_train["ACT"] != x_train["PRED"]]

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,ACT,PRED
5,45333,14561,117951,117952,118008,118568,118568,19721,118570,0,1
41,74310,49521,117961,118300,118301,119849,235245,118638,119851,0,1
55,78106,50613,117916,118150,118810,118568,159905,19721,118570,0,1
66,35043,14800,117961,117962,118352,118784,117906,290919,118786,0,1
68,15716,18073,118256,118257,118623,118995,286106,292795,118997,0,1
91,22956,3967,117961,118052,118706,118321,117906,290919,118322,0,1
102,36480,15510,118169,118170,118171,130479,185784,119784,130481,0,1
112,80501,19717,117961,117962,118352,118321,117906,290919,118322,0,1
122,23096,29762,117961,118052,120398,122067,130238,118424,122069,0,1
156,77977,3811,120864,121013,118395,118396,269406,118398,118399,0,1


In [41]:
x_train = df_train.loc[:, ~df_train.columns.isin(["ACTION"])]
y_train = df_train["ACTION"]
x_test = df_test.loc[:, ~df_test.columns.isin(["id"])]
y_test = df_train["ACTION"]

In [42]:
df_test.head()

,id,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,32642,7792,118573,118574,117945,136261,128463,292795,119082
1,2,4696,14638,117961,118343,118514,118321,289122,255851,118322
2,3,22662,1760,118887,118888,120171,118396,255118,118398,118399
3,4,75078,7576,117961,118052,120671,118321,117906,257051,118322
4,5,39879,55668,117902,118041,117945,135951,134458,19776,119082


In [43]:
clf = RandomForestClassifier(n_estimators=10, max_depth=5)
clf.fit(x_train,y_train)
y_pred_test = clf.predict(x_test)

In [44]:
ID = df_test["id"]
#ID = pd.DataFrame(ID).reset_index()
ID = pd.DataFrame(ID)

In [45]:
ACTION = pd.DataFrame(y_pred_test)

In [46]:
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.feature_selection import chi2
selector = SelectKBest(chi2, k=6)
X_new=selector.fit_transform(x_train, y_train)
mask = selector.get_support(indices=True)
colname = x_train.columns[mask]
colname

Index(['MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_TITLE', 'ROLE_FAMILY_DESC',
       'ROLE_FAMILY', 'ROLE_CODE'],
      dtype='object')